# Coursera Capstone Project 
# Week 3 Part 1 and 2

### by Sushant Thotakura

Adding the longitude and latitude data for the postal codes

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')


Libraries imported.


In [2]:
#importing the table from the webpage into a dataframe
import requests
from bs4 import BeautifulSoup
url_html='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=next&oldid=942655364'
req = requests.get(url_html)
soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df = pd.read_html(str(table))
neighborhood = pd.DataFrame(df[0])


In [3]:
neighborhood.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Removing Not assigned datas for Borough

In [4]:
#removing the not assigned values from the data frame
toronto_df = neighborhood[neighborhood.Borough != "Not assigned"].reset_index(drop=True)
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


### Grouping the neighborhoods in the same Borough

In [5]:
#grouping the neighborhoods in the same borough
toronto_grouped = toronto_df.groupby(["Postcode","Borough"], as_index=False).agg(lambda x:",".join(x))
toronto_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### For Not assigned Neighborhood data, make the value the same as Borough

In [6]:
# for neighborhood = not assigned, make the value the same as Burough

for index, row in toronto_grouped.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]
        
toronto_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### create the example data frame from the instructions

In [7]:
#create the example data frame 
column_names = ["Postcode", "Borough", "Neighbourhood"]
example_df = pd.DataFrame(columns=column_names)

example_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for n in example_list:
    example_df = example_df.append(toronto_grouped[toronto_grouped["Postcode"] == n], ignore_index=True)
    
example_df

,Postcode,Borough,Neighbourhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens,Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale,Wexford"
7,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf..."


### Print the shape of the data frame

In [8]:
#print the shape of the data frame

toronto_grouped.shape

(103, 3)

### Load the longitutde and latitude from the csv file on coursera

In [9]:
# load the longitude and latitude data from the csv file
coor = pd.read_csv("http://cocl.us/Geospatial_data")
coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Rename the column names to match original data

In [10]:
#rename the postal code column name to match the original data
coor.rename(columns={"Postal Code":"Postcode"}, inplace = True)
coor.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge the two tables

In [11]:
#merge the longitude and latitude data with the original data frame
toronto_df = toronto_grouped.merge(coor, on="Postcode", how="left")
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Check the required data matches with the example

In [13]:
#check to match the example data
columns_names_loglat = ["Postcode", "Borough", "Neighbourhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=columns_names_loglat)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for i in test_list:
    test_df = test_df.append(toronto_df[toronto_df["Postcode"]==i], ignore_index=True)
    
test_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Maryvale,Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420
